In [ ]:
# Per treballar amb COCO necessitam instal·lar aquesta llibreria.
# Cada cop que iniciam l'entorn s'ha d'executar

!pip install pycocotools

# UNET

## Cárrega de dades

En primer lloc hem de carregar les dades necessàries. Com hem comentat a l'enunciat farem feina amb la versió 2014 de [COCO](https://cocodataset.org#home)

In [6]:
import math
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from skimage.transform import resize

import tensorflow as tf

from pycocotools.coco import COCO
import pycocotools.mask as maskUtils

Al vostre google drive heu de tenir una carpeta anomenada `unet_aa`.  En aquesta
carpeta hi heu de **descomprimir** la informació del dataset [enllaç](http://images.cocodataset.org/annotations/annotations_trainval2014.zip) . Si l'enllaç no funciona podeu anar a la secció de descarregues i cercar _2014 Train/Val annotations [241MB]_.

També heu de crear una carpeta anomenada _license_.

El següent codi, permet connectar el _notebook_ de colab amb aquesta carpeta. En executar-ho us demanarà permís explicit per accedir-hi mitjançant un pop-up.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My Drive/unet_aa

**Les categories que podem emprar del COCO són:** 
person, bicycle, car, motorcycle, airplane, bus, train, truck, boat, traffic light, fire hydrant stop sign, parking meter, bench, bird, cat, dog, horse, sheep, cow, elephant, bear, zebra, giraffe, backpack, umbrella, handbag, tie, suitcase, frisbee, skis, snowboard, sports, ball, kite, baseball, bat, baseball, glove, skateboard, surfboard, tennis racket, bottle, wine, glass cup, fork, knife, spoon, bowl, banana, apple, sandwich, orange, broccoli, carrot, hot dog, pizza, donut, cake, chair, couch, potted, plant, bed, dining table, toilet, tv, laptop, mouse, remote, keyboard, cell phone, microwave, oven, toaster, sink, refrigerator, book, clock, vase, scissors, teddy bear, hair drier, toothbrush.

A continuació teniu un **generador** que ens permet tenir un conjunt d'entrenament i un conjunt de test. Aquests generadors tenen tres paràmetres: la categoría, la mida del _batch_ i un _booleà_ que ens indica si es el generador del conjunt d'entrenament o de test. 

In [7]:
class MyGeneration(tf.keras.utils.Sequence):
  
  def __init__(self, category, batch_size, train=True):

    dataDir = ''
    dataType = 'val2014'
    annFile = '{}annotations/instances_{}.json'.format(dataDir,dataType)

    self.coco = COCO(annFile)
    
    self.batch_size = batch_size
    self.cat = category
    self.catIds = self.coco.getCatIds(catNms=['dog']);
    self.imgIds = self.coco.getImgIds(catIds=self.catIds);
    
    if train:
      self.imgIds = self.imgIds[:int(len(self.imgIds)*0.75)]
    else: 
      self.imgIds = self.imgIds[int(len(self.imgIds)*0.75):]
  
  def __len__(self):  
    return math.ceil(len(self.imgIds) / self.batch_size)
    
  def __getitem__(self, idx):
    
    batch_x = self.coco.loadImgs(self.imgIds[idx * self.batch_size:(idx + 1) * self.batch_size])
    
    self.annIds = self.coco.getAnnIds(imgIds=self.imgIds[idx * self.batch_size:(idx + 1) * self.batch_size],
                                 catIds=self.catIds, iscrowd=False)
    
    pre_batch_y = []

    for img_id in self.imgIds[idx * self.batch_size:(idx + 1) * self.batch_size]:

      annIds      = self.coco.getAnnIds(imgIds=img_id, catIds=self.catIds, iscrowd=False)
      annotations = self.coco.loadAnns(annIds)
      mask        = self.coco.annToMask(annotations[0])
      
      for ann in annotations[1:]:
        mask += self.coco.annToMask(ann)
      mask = resize(mask, (256, 256, 1))
      mask[mask>0] = 1

      pre_batch_y.append(mask)


    return (np.array([resize(imread(file_name['coco_url']), (256, 256)) for file_name in batch_x]), 
            np.array([mask for mask in pre_batch_y]))

Finalment només ens queda crear els dos generadors:

In [ ]:
gen_train = MyGeneration('dog', 10)
gen_test = MyGeneration('dog', 10, False)


### Importar les llibreries

In [9]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as keras

## Definició del model


In [10]:
def unet(input_size = (256, 256, 3), output_channels = 1):

    dilation_rate = 1 # No modificar
    inputs = Input(input_size)
    conv1 = Conv2D(16, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(inputs)
    conv1 = Conv2D(16, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format='channels_last')(conv1)

    conv2 = Conv2D(32, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(pool1)
    conv2 = Conv2D(32, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format='channels_last')(conv2)

    conv3 = Conv2D(64, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(pool2)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(conv3) 
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format='channels_last')(conv3)

    conv4 = Conv2D(128, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(pool3) 
    conv4 = Conv2D(128, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2), data_format='channels_last')(conv4)

    conv5 = Conv2D(256, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(pool4)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(conv5)  
    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=3)
    
    # TODO your code here
    
    return model


In [ ]:
model = unet()   
# TODO: compilar

# TODO: Entrenar


## Avaluació

Teniu codi per poder visualitzar alguns resultats. Podeu modificar aquest codi al vostre gust

In [ ]:
# Obtenim el primer batch del conjunt d' avaluació
for test in gen_test:
  break

# Realitzam la predicció
prediccio = model.predict(test[0])

# Mostram els resultats
n_images = len(prediccio)

figure = plt.figure(1, figsize=(9, 9))

for i in range(n_images):

  ax = plt.subplot(n_images/2, n_images//2, i + 1)
  ax.imshow(test[0][i,:,:,:])
  ax.set_xticks(())
  ax.set_yticks(())

figure = plt.figure(2, figsize=(9, 9))

for i in range(n_images):

  ax = plt.subplot(n_images/2, n_images//2, i + 1)
  predit = prediccio[i][:,:,0]
  predit[predit > 0.3] = 255
  predit[predit < 255] = 0 
  predit = predit.astype(np.uint8)
  ax.imshow(predit) #test[1][i,:,:,0]*255)
  ax.set_xticks(())
  ax.set_yticks(())